In [26]:
import os

from dotenv import load_dotenv
load_dotenv(dotenv_path="C:/Users/Lenovo/Desktop/LANGCHAIN/.env", override=True)

True

In [15]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.llms import HuggingFaceHub

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import ChatPromptTemplate

from langchain.chains import retrieval_qa

### EXTRACT, LOAD, SPLIT

In [2]:
pdf_dir_loader = PyPDFDirectoryLoader("C:/Users/Lenovo/Desktop/LANGCHAIN/huggingface/us_census/")

pdf_dir_docs = pdf_dir_loader.load()

pdf_dir_docs_split = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(pdf_dir_docs)

pdf_dir_docs_split[0]

Document(metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'C:\\Users\\Lenovo\\Desktop\\LANGCHAIN\\huggingface\\us_census\\acsbr-015.pdf', 'total_pages': 18, 'page': 0, 'page_label': '1'}, page_content='Health Insurance Coverage Status and Type \nby Geography: 2021 and 2022\nAmerican Community Survey Briefs\nACSBR-015\nIssued September 2023\nDouglas Conway and Breauna Branch\nINTRODUCTION\nDemographic shifts as well as economic and govern-\nment policy changes can affect people’s access to \nhealth coverage. For example, between 2021 and 2022, \nthe labor market continued to improve, which may \nhave affected private coverage in the United States \nduring that time.\n1 Public policy changes inc

In [3]:
total_docs = len(pdf_dir_docs_split)
total_docs

316

### EMBED

In [8]:
hugging_face_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5",
                                   model_kwargs={'device': 'cpu'},
                                   encode_kwargs={'normalize_embeddings': True})

### STORE IN VECTOR DB

In [10]:
vectorstore = FAISS.from_documents(pdf_dir_docs_split[:100], hugging_face_embeddings)

In [12]:
similarity_search = vectorstore.similarity_search("What is the health insurance coverage?")
similarity_search[0].page_content


'private health insurance as a plan provided through an employer \nor a union, coverage purchased directly by an individual from an \ninsurance company or through an exchange (such as healthcare.\ngov), or coverage through TRICARE. Public insurance coverage \nincludes federal programs (such as Medicare, Medicaid, and the \nChildren’s Health Insurance Program or CHIP), individual state \nhealth plans, and CHAMPVA (Civilian Health and Medical Program \nat the Department of Veterans Affairs), as well as care provided \nby the Department of Veterans Affairs. In the ACS, people are \nconsidered insured if they were covered by any of these types \nof health insurance at time of interview. People are considered \nuninsured if they were not covered by any of these types of health \ninsurance at time of interview or if they only had coverage through \nthe Indian Health Service (IHS), as IHS coverage is not considered \ncomprehensive.'

### RETRIEVER

In [27]:
llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.1", 
                     model_kwargs={"temperature": 0.1, "max_length": 500})

In [28]:
query = "What is the health insurance coverage?"

llm.invoke(query)

AttributeError: 'InferenceClient' object has no attribute 'post'

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", 
                                     search_kwargs={"k": 3})

retriever.invoke("What is the health insurance coverage?")

[Document(id='3f05dc14-479b-4b43-9e6f-f7f684ae9719', metadata={'producer': 'Adobe PDF Library 17.0', 'creator': 'Adobe InDesign 18.2 (Windows)', 'creationdate': '2023-09-09T07:52:17-04:00', 'author': 'U.S. Census Bureau', 'keywords': 'acsbr-015', 'moddate': '2023-09-12T14:44:47+01:00', 'title': 'Health Insurance Coverage Status and Type by Geography: 2021 and 2022', 'trapped': '/false', 'source': 'C:\\Users\\Lenovo\\Desktop\\LANGCHAIN\\huggingface\\us_census\\acsbr-015.pdf', 'total_pages': 18, 'page': 1, 'page_label': '2'}, page_content='private health insurance as a plan provided through an employer \nor a union, coverage purchased directly by an individual from an \ninsurance company or through an exchange (such as healthcare.\ngov), or coverage through TRICARE. Public insurance coverage \nincludes federal programs (such as Medicare, Medicaid, and the \nChildren’s Health Insurance Program or CHIP), individual state \nhealth plans, and CHAMPVA (Civilian Health and Medical Program \nat